In [2]:
from ssl import CERT_NONE
from pprint import pprint

from nostr.event import Event
from nostr.key import PrivateKey
from nostr.relay_manager import RelayManager

In [5]:
# Setup Alice's keys
privkey: PrivateKey = PrivateKey().from_nsec(ALICE)
print(privkey.public_key.bech32())

npub1tqh0wyekvvhfc82ux52gd4tahusvfx7wt2789vtvsny3jed5x0gqht6tun


In [37]:
def connect_to_relays(relays:list[str]) -> RelayManager:
    relay_manager = RelayManager()
    for r in relays:
        relay_manager.add_relay(r)
    relay_manager.open_connections({"cert_reqs": CERT_NONE})
    return relay_manager

In [45]:
# Connect to relays
relay_manager = connect_to_relays(
    relays=[
        RELAY,
        "wss://relay.damus.io",
        "wss://relay.snort.social",
        "wss://nos.lol"
    ]
)

pprint(relay_manager.relays)

{'wss://nor.st': <nostr.relay.Relay object at 0x107e9ad50>,
 'wss://nos.lol': <nostr.relay.Relay object at 0x107e98410>,
 'wss://relay.damus.io': <nostr.relay.Relay object at 0x107e9b7d0>,
 'wss://relay.snort.social': <nostr.relay.Relay object at 0x107e99c50>}


In [8]:
def create_badge(public_key:str, kind:int=30009, content:str='', tags:list=[],) -> Event:
    return Event(
        public_key=public_key,
        kind=kind,
        content=content,
        tags=tags,
    )

In [9]:
# Create cake card badges
cake_card = create_badge(
    public_key=privkey.public_key.hex(),
    tags = [
        ['d', 'cake-card-01'],
        ['name', 'Cake Card $21'],
        ['description', 'Good for $21 of Cake'],
        ['image', 'https://icons.iconarchive.com/icons/iconarchive/fat-sugar-food/256/Cake-icon.png', '1024x1024'],
        ['thumb', 'https://icons.iconarchive.com/icons/iconarchive/fat-sugar-food/256/Cake-icon.png', '256x256']
    ],
)

cake_card_consumed = create_badge(
    public_key=privkey.public_key.hex(),
    tags = [
        ['d', 'cake-card-consumed-01'],
        ['name', 'Cake Card Consumed $21'],
        ['description', '$21 Cake Card was Consumed'],
        ['image', 'https://icons.iconarchive.com/icons/sonya/swarm/256/Poop-icon.png', '1024x1024'],
        ['thumb', 'https://icons.iconarchive.com/icons/sonya/swarm/256/Poop-icon.png', '256x256']
    ],
)

In [10]:
# Sign badge creation events
privkey.sign_event(cake_card)
privkey.sign_event(cake_card_consumed)

print(cake_card.signature)
print(cake_card_consumed.signature)

36964b46946ed3ef29eab70a2a7956f9d393c649e224ab16d00e87bf042f9daa1ebb9cd2a7335e06fc9feede9c119ef2e62ac570d769bdd711fe694b0bc36732
f269cc81ecbf15d892845161de7578eb74cf876e62718bf03086b6e5bb99cfaa867f293148c656f6ba888529a5a432b12745fd13ee85dd25b11b072520e130f1


In [11]:
# Publish badge creation events
relay_manager.publish_event(cake_card)
relay_manager.publish_event(cake_card_consumed)

print(cake_card.id)
print(cake_card_consumed.id)

1801f9b8e498083b208b4f25c3877f1af830d2d594b0f21dc78e4ad1c8345933
d19820d0a8012e2db1cfe9f6e612e04bb1cd98f17c544cbb0dde7bb044c36101


In [41]:
def assign_badge(public_key:str, assign_to:str, badge_d:str,
                 kind:int=8, content:str='', tags:list=[]) -> Event:
    return Event(
        public_key=public_key,
        kind=kind,
        content=content,
        tags=[
            ['a', f'30009:{public_key}:{badge_d}'],
            ['p', f'{assign_to}', 'wss://relay.snort.social']
        ],
    )

In [42]:
# Create badge assignment event
assign_cake = assign_badge(public_key=privkey.public_key.hex(), assign_to=BOB, badge_d='cake-card-01')

print(assign_cake.id)

aaf6700eeca1babc03a5e30efc1770f10f88a40f6a466dbc3a755f9766489cdb


In [43]:
# Sign badge assignment event
privkey.sign_event(assign_cake)

In [46]:
# Publish badge assignment event
relay_manager.publish_event(assign_cake)

In [47]:
# Create badge consumption assignment event
assign_cake_consumed = assign_badge(public_key=privkey.public_key.hex(), assign_to=BOB, badge_d='cake-card-consumed-01')

print(assign_cake_consumed.id)

f940746d8d3c5f4895d3e0aead94afdfcb3992cc438cfd8072c65797efb4e62f


In [48]:
# Sign badge assignment event
privkey.sign_event(assign_cake_consumed)

In [49]:
# Publish badge assignment event
relay_manager.publish_event(assign_cake_consumed)

In [4]:
RELAY: str = 'wss://nor.st'
ALICE: str = 'nsec1mp95lpaxpd50dvvh7cq0azasvdl8cd0qgu55qtlmz3he25wj2t2q52lx9k'
BOB: str = '4b9c064b4863cc0d02d794f4fff9e7793ddadd93175623987642fe073b5d5dd2'